## Импорт и определение библиотек

In [1]:
# отключение параллелизма
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
# ставим разные пакеты оттуда
!pip install langchain langchain-community langchain_huggingface unstructured pypandoc -q
!pip install faiss-cpu -q
!pip install langchain-ollama -q

In [13]:
# импортируем библиотеки
from langchain.document_loaders import WebBaseLoader, PyPDFLoader, UnstructuredEPubLoader
from langchain_core.documents import Document
from langchain.prompts import PromptTemplate
from langchain.tools import Tool
from langchain.agents import initialize_agent
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFaceEmbeddings
import tqdm as notebook_tqdm
from langchain_community.vectorstores import FAISS
from langchain.llms import Ollama
from langchain_ollama import OllamaLLM

# Готовим коктейль с помощью RAG
## Чутка `embeddings` с `HuggingFace` для оснастки

In [4]:
# Чтение токена из файла
def read_token(file_path): 
    with open(file_path, 'r') as f:
        for line in f:
            if line.startswith("HUGGINGFACEHUB_API_TOKEN"):
                return line.split('=')[1].strip()
    raise ValueError("Token not found in the specified file.")

# Загрузка токена
#hf_token = read_token('../utils.key')

# Формирование эмбеддингов
hf_embeddings_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",  
    #"cointegrated/LaBSE-en-ru", 
    model_kwargs={"device": "cpu"} 
)

/home/elijah/Documents/LLM/cocktail_qr/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Можно пропустить (если используете наполненную чашу векторного хранилища)
### Немного `Document Loader`

In [17]:
# Загрузка EPUB файла от бармена-литературоведа
loader = UnstructuredEPubLoader('../docs/Federle.epub', show_progress=True)
doc_epub_mistral = loader.load()

# смотрим, скока получилось загрузить
print('pages count:', len(doc_epub_mistral))
print('max chars on page:', max([len(page.page_content) for page in doc_epub_mistral]))

pages count: 1
max chars on page: 107261


In [ ]:
# Загружаем книгу от крутого бармена
#loader = WebBaseLoader(url)
"""loader = PyPDFLoader('../docs/Bortnik_1000.pdf')
data = loader.load()

print('pages count:', len(data))
print('max chars on page:', max([len(page.page_content) for page in data]))"""

# на тот случай, если решим загружать подобную литературу пачкой pdf
"""
#!pip install unstructured "unstructured[pdf]" -q
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader("../docs", glob="**/*.pdf", show_progress=True)
data = loader.load()"""

### Исполним `Text splitters` и разобьём на чанки

In [37]:
from langchain.text_splitter import TextSplitter
from typing import List
    
class RecipeSplitter(TextSplitter):
    def __init__(self):
        super().__init__(chunk_size=1000, chunk_overlap=200)
    
    def split_text(self, text: str) -> List[str]:
        # Убираем все пустые строки и лишние пробелы
        cleaned_text = "\n".join([line.strip() for line in text.split("\n") if line.strip()])
        
        # Разделяем текст на рецепты по маркеру "НАЗВАНИЕ_РЕЦЕПТА\nАВТОР"
        chunks = []
        current_chunk = []
        lines = cleaned_text.split("\n")
        
        i = 0
        while i < len(lines):
            line = lines[i]
            # Проверяем начало нового рецепта
            if i + 1 < len(lines) and lines[i+1].isupper() and "(" in lines[i+1] and ")" in lines[i+1]:
                if current_chunk:
                    chunks.append("\n".join(current_chunk))
                    current_chunk = []
                current_chunk.append(line)
                current_chunk.append(lines[i+1])
                i += 2
                continue
            
            current_chunk.append(line)
            i += 1
        
        if current_chunk:
            chunks.append("\n".join(current_chunk))
        
        return chunks
    
splitter = RecipeSplitter()

In [43]:
split_documents = []
for page in doc_epub_mistral:
    split_documents += splitter.create_documents([page.page_content], metadatas=[page.metadata])

print('Количество чанков:', len(split_documents))

Количество чанков: 73


### Наполним чашу `Vector Store`

In [44]:
# Если нужно сохранить новый документ в базу
try:
    db_embed = FAISS.from_documents(
            split_documents, 
            hf_embeddings_model
        )
except Exception as e:
    raise RuntimeError(f"Ошибка при создании базы данных FAISS: {e}")


# Сохраняем векторную базу локально
db_embed.save_local("../data/faiss_db_epub_mistral")

## Нужно запустить (если используете готовое векторное хранилище)
### Испьём чашу `Vector Store`

In [5]:
# Если нужно изъять из уже сохранённого документа из базы
# Загрузка векторной базы данных из локального файла
db_embed = FAISS.load_local(
        "../data/faiss_db_epub_mistral", 
        hf_embeddings_model,
        allow_dangerous_deserialization = True
    )

In [9]:
query = "в какой коктейль входит ржаной виски"
results = db_embed.similarity_search(query, k=5)
for result in results:
    print(result.page_content)

Старик и more виски
СТАРИК И МОРЕ (1952)
ЭРНЕСТ ХЕМИНГУЭЙ
Удостоенная Пулитцеровской премии аллегорическая повесть «Старик и море» вышла в свет под занавес писательской карьеры Хемингуэя, полной наград, дифирамбов – и алкоголя. Завязка проста (и знакома читателям «Моби Дика» и почитателям «Моби Дринка»): старик в исступлении решает во что бы то ни стало взять верх над огромной рыбиной. Из своей эпической трехдневной битвы, в ходе которой гигантский марлин загарпунен, привязан к корме, но – вот так-то, старина! – сожран акулами, старик выходит измученным, но победителем. Вооружимся же и мы непременными рыбацкими атрибутами – виски и приманкой – и смешаем напиток, с которым море по колено:
60 мл виски (например, Benchmark);
1 банка (0,33 л) лимонно-лаймовой газировки (например, Sprite);
кумкват для украшения.
Находим свой самый большой хайбол. Бросаем на дно кубики льда, вливаем виски и газировку. Теперь берем приманку под названием воблер (в виде рыбки с крючком), промываем с мылом, цеп

# Месье системный промт

In [10]:
# Системный промпт
system_prompt = """Ты — бармен. Используй данные из контекста. 
Формат ответа:
1. Название коктейля (с метафорой)
2. Состав:
- Ингредиенты (объем)
3. Способ приготовления (шаги с креативными деталями).

Стиль: {style}. """

# Промпт для LangChain
qa_prompt = PromptTemplate(
    template=system_prompt + "\nКонтекст: {context}\nВопрос: {question}\nОтвет:",
    input_variables=["style", "context", "question"]
)

## Полирнём `Retriver` и инициализирует `llm` для вкуса

In [15]:
# Инициализация модели
llm = OllamaLLM(
    model="mistral:instruct",
    temperature=0.2,          # Для креативности
    num_predict=512,         # Увеличенная длина ответа
    stop=["\\n\\n", "###"],   # Стоп-токены
    top_k=40
)

In [85]:
# Модифицируем параметры ретривера
retriever = db_embed.as_retriever(
    search_type="mmr",              # Maximal Marginal Relevance
    search_kwargs={
        "k": 5,                     # Уменьшаем количество возвращаемых документов
        "fetch_k": 20,              # Уменьшаем общий пул для поиска
        #"lambda_mult": 0.5          # Баланс между релевантностью и разнообразием
    }
)

# Инициализация цепочки RetrievalQA
retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,  # Ваш ретривер (например, FAISS)
    #chain_type_kwargs={"prompt": qa_prompt},
    return_source_documents=True
)

In [ ]:
"""# Создание кастомной цепочки RetrievalQA
from langchain.chains import LLMChain
from langchain.chains.retrieval_qa.base import BaseRetrievalQA
from typing import Any, Dict, List
from langchain.schema import Document


class CustomRetrievalQA(RetrievalQA):
    def _call(self, inputs, run_manager=None):
        question = inputs["query"]
        style = inputs["style"]
        # Используем метод get_relevant_documents вместо _get_docs
        docs = self.retriever.get_relevant_documents(question, callbacks=run_manager.get_child())
        context = "\n".join([doc.page_content for doc in docs])
        combine_inputs = {
            "style": style,
            "context": context,
            "question": question
        }
        return self.combine_documents_chain.run(combine_inputs, callbacks=run_manager.get_child())

# Инициализация цепочки RetrievalQA
combine_documents_chain = LLMChain(llm=llm, prompt=qa_prompt)
retrieval_qa = CustomRetrievalQA(
    retriever=retriever,
    combine_documents_chain=combine_documents_chain,
    return_source_documents=True
)"""

# Миксуем рецепт коктейля `agent` 00х

In [87]:
# Создание инструмента
agent_tools = [
    Tool(
        name="Cocktail Recipe Finder",
        func=retrieval_qa.run,
        description="Используй этот инструмент, чтобы найти рецепты коктейлей по заданным ингредиентам. Вводи список ингредиентов, и инструмент вернёт подходящие рецепты. Пример: ['сок', 'виски']."
    )
]

# создание агента
agent = initialize_agent(
    tools=agent_tools, 
    llm=llm, 
    agent="conversational-react-description",       #"zero-shot-react-description", "plan-and-execute"
    verbose=True,
    system_prompt=system_prompt                     # не забыть указать системный промт для вкуса
)

# Задаём общие параметры, после чего коктейль готов
## Вначале коктейль

In [88]:
# Объявление функции на ввод данных от пользователя
def get_user_input():
    """
    Запрашивает у пользователя ингредиенты для поиска рецептов.
    """
    user_input = input("Введите ингредиенты, разделенные запятыми: ")
    ingredients = [ingredient.strip() for ingredient in user_input.split(",")]
    return ingredients


# Обработка ответа
def format_response(response):
    if "не знаю" in response.lower() or "не найдено" in response.lower():
        return "К сожалению, я не нашёл коктейлей с этими ингредиентами. Попробуйте другие ингредиенты."
    else:
        return response
    

# Функция выбора стиля ответа
def choose_style(styles):
    chosen_style = input("Выберите номер стиля (по умолчанию 4): ") or "4"
    return styles.get(chosen_style, styles["4"])#["name"]

# Генерация нового ответа в выбранном стиле 
def apply_style(response: str, style: str) -> str:
    """Применяет выбранный стиль к ответу."""
    prompt = f"""
    Перепиши этот рецепт в стиле «{style}». Сохрани структуру (название, состав, шаги). 
    Текст для стилизации: {response}
    """
    return llm.invoke(prompt)

In [89]:
def main():
    # Стилевые инструкции с акцентом на общие характеристики
    style_instructions = {
        "1": {
            "name": "Космический ужас",
            "description": "Используй слова: 'древний', 'тьма', 'запретный ритуал'.",
            "params": {"temperature": 0.8, "top_p": 0.95}
        },
        "2": {
            "name": "Гопнический жаргон",
            "description": "Сленг: 'замути', 'вмазать', 'движняк'.",
            "params": {"temperature": 0.7, "top_p": 0.9}
        },
        "3": {
            "name": "Поэтический стиль",
            "description": "Метафоры: 'танец вкусов', 'шепот льда', 'рассвет в бокале'.",
            "params": {"temperature": 0.75, "top_p": 0.85}
        },
        "4": {
            "name": "Формальный",
            "description": "Технический язык без эмоций.",
            "params": {"temperature": 0.3, "top_p": 0.8}
        }
    }
    
    while True:
        ingredients = get_user_input()
        if not ingredients:
            print("Вы не ввели ингредиенты. Попробуйте снова.")
            continue
        
        # Выбор стиля
        chosen_style = choose_style(style_instructions)
        style_name = chosen_style["name"]
        
        # Базовый запрос
        ingredients_str = ", ".join(ingredients)
        user_prompt = f"""
        Найди рецепты коктейлей по ингредиентам: {ingredients_str}
        Учти соответствие с системным промтом и стилистикой ответа.
        """
        
        # Вызов RetrievalQA с передачей стиля
        response = retrieval_qa.invoke({
            "style": style_name,
            "query": user_prompt
        })
        
        original_response = response["result"]
        
        # Применение стиля
        styled_response = apply_style(original_response, style_name)
        
        # Постобработка
        styled_response = styled_response.strip()
        print(styled_response)
        break
    
    return styled_response

if __name__ == "__main__":
    # Сохраняем результат в переменную
    recipe = main()

1. Southern Comfort Cocktail (Southern Comfort et Juice de Pamplemousse)

Dans un shaker rempli de glace, combinez 2 onces de Southern Comfort et une once de jus de pamplemousse frais. Agitez vigoureusement jusqu'à ce que le cocktail soit refroidi. Versez-le dans une verrine froide remplie de glace pilée. Décorez avec un tranche de pamplemousse.

2. Sour à la Whiskey de Rye (Whiskey de Rye, Jus de Citron, Jus de Pamplemousse)

Dans un shaker rempli de glace, combinez 2 onces de whiskey de rye, une once de jus de citron frais, et une demie once de jus de pamplemousse frais. Agitez vigoureusement jusqu'à ce que le cocktail soit refroidi. Versez-le dans une verrine froide remplie de glace pilée. Décorez avec un tranche de citron ou de pamplemousse.

3. Old Fashionné à la Pamplemousse (Whiskey de Rye, Jus de Pamplemousse, Sirop de Sucre)

Dans un verre à cocktail rempli de glace, combinez 2 onces de whiskey de rye, une demie once de sirop de sucre, et deux onces de jus de pamplemousse frai

In [90]:
def main():
    # Определение стилей
    styles = {
        "1": "стиль космического ужаса и хтонического мрака Говарда Ф. Лавкрафта",
        "2": "гопническо-быдляцкий жаргон",
        "3": "экспериментальный стиль нарезок Уильяма Берроуза",
        "4": "стиль без изменений"  # Стиль по умолчанию
    }
    
    while True:
        ingredients = get_user_input()
        if not ingredients:
            print("Вы не ввели ингредиенты. Попробуйте снова.")
            continue
        
        # Преобразуем массив ингредиентов в строку
        ingredients_str = ", ".join(ingredients)
        
        # Формируем текстовый запрос
        user_prompt = (
            """Напиши, какие коктейли можно изготовить из представленных ниже ингредиентов. 
            Для каждого коктейля опиши подробный способ приготовления и укажи пропорции. 
            Отвечай строго на русском языке, используя чёткий и лаконичный формат. 
            Не добавляй лишнюю информацию, которая не относится к рецепту. 
            Ингредиенты: {ingredients}""".format(ingredients=ingredients_str)
        )
        response = retrieval_qa.invoke({"query": user_prompt})  
        
        # Получаем ответ без вывода его на экран
        original_response = response["result"]
        # Выбор стиля
        style = choose_style(styles)
        prompt = PromptTemplate(input_variables=['output_text', 'style'],
                                template='''Добро пожаловать в поиск коктейлей!\nПерепиши этот текст в заданном стиле: {output_text}\nСтиль: {style}.\nРезультат:\n''')
        
        # Генерация текста с использованием оригинального ответа и выбранного стиля
        styled_prompt = prompt.format(output_text=original_response, style=style)
        
        # Применение выбранного стиля к оригинальному ответу
        styled_response = llm.invoke(styled_prompt)#.content
        print(styled_response)
        break
    
    return styled_response

if __name__ == "__main__":
    # Сохраняем результат в переменную
    recipe = main()

1. Грейпфрутовый Old Fashioned
     Способ приготовления: Залить стакан полностью грейпфрутовым соком. Добавить ржаный виски на высоту 3/4 стакана. Мешать необходимо, но не слишком сильно.
     Пропорции: 1 часть грейпфрутового сока, 3/4 частей ржаного виски.

  2. Грейпфрутовый Old Fashioned (вариация)
     Способ приготовления: Залить стакан полностью грейпфрутовым соком. Добавить ржаный виски на высоту 1/2 стакана. Мешать необходимо, но не слишком сильно.
     Пропорции: 1 часть грейпфрутового сока, 1/2 частей ржаного виски.


# Реюинион

In [1]:
from langchain.text_splitter import TextSplitter
from typing import List
    
class RecipeSplitter(TextSplitter):
    def __init__(self):
        super().__init__(chunk_size=1000, chunk_overlap=0)
    
    def split_text(self, text: str) -> List[str]:
        # Убираем все пустые строки и лишние пробелы
        cleaned_text = "\n".join([line.strip() for line in text.split("\n") if line.strip()])
        
        # Разделяем текст на рецепты по маркеру "НАЗВАНИЕ_РЕЦЕПТА\nАВТОР"
        chunks = []
        current_chunk = []
        lines = cleaned_text.split("\n")
        
        i = 0
        while i < len(lines):
            line = lines[i]
            # Проверяем начало нового рецепта
            if i + 1 < len(lines) and lines[i+1].isupper() and "(" in lines[i+1] and ")" in lines[i+1]:
                if current_chunk:
                    chunks.append("\n".join(current_chunk))
                    current_chunk = []
                current_chunk.append(line)
                current_chunk.append(lines[i+1])
                i += 2
                continue
            
            current_chunk.append(line)
            i += 1
        
        if current_chunk:
            chunks.append("\n".join(current_chunk))
        
        return chunks

In [ ]:
# Пример текста
text = """
Рожь и предубеждение
ГОРДОСТЬ И ПРЕДУБЕЖДЕНИЕ (1813)
ДЖЕЙН ОСТИН
Роман благородных нравов XIX века подарил Джейн Остин шанс еще при жизни (хоть и недолгой – всего 41 год) вкусить свои пять минут славы. «Гордость и предубеждение» повествует о попытках семейства Беннет выдать замуж своих дочерей. От лица одной из пяти девиц и ведется рассказ. Увы, Элизабет – в одной из экранизаций блеснувшая выразительными скулами Киры Найтли – та еще язва, и ее жажда припечатать всех и каждого чуть ли не затмевает ее трепетное чувство к мистеру Дарси, джентльмену с завидным самомнением (зато ну очень богатому!). Впрочем, все кончается восхитительной двойной свадьбой. Вот и мы сосватаем две сильные натуры – терпкую пряность ржи и жизнеутверждающую горечь грейпфрута – и на славу погуляем на нашем импровизированном торжестве:


90 мл грейпфрутового сока;

50 мл ржаного виски.


Ингредиенты выливаем в стакан рокс поверх кубиков льда, размешиваем в ритме «сердце вскачь». Хотим подчеркнуть, юные леди, мы нисколечко не предубеждены против замужества, а лишь призываем вас уяснить главное: чтобы почувствовать себя королевой, не нужно и дворца (как, впрочем, и короля).


Кофейный ликер без меры
ЛЮБОВЬ ВО ВРЕМЯ ХОЛЕРЫ (1985)
ГАБРИЭЛЬ ГАРСИА МАРКЕС
Зачем вы, девочки, докторов заграничных любите? Отдашь такому сердце – и полвека промаешься, прежде чем найти то самое, что доктор прописал. Вот она какая, романтика по Маркесу: юноша и девушка загорелись друг к другу пылкой страстью, но девица возьми да выбери себе в мужья доктора – а возлюбленному от ворот поворот дала, крути, мол, в отместку с кем хочешь. Но над истинной страстью время не властно! И однажды, «пятьдесят один год, девять месяцев и четыре дня спустя» (надо же было высчитать!), после кончины Супруга-Номер-Один, те двое воссоединяются. Жемчужина колумбийской литературы, роман Гарсиа Маркеса заслуживает нашего салюта – сладкого, как истинная любовь, и пряного, как неувядающая страсть:


30 мл светлого рома;

15 мл кофейного ликера (например, «Калуа»);

60 мл нежирных сливок;

щепотка молотой корицы или мускатного ореха.


В стакан рокс укладываем лед и поливаем его ромом и кофейным ликером. Добавляем сливки и приправу по вкусу. А теперь опрокидываем нашу гремучую смесь во славу истинной страсти – даже если заморского врача вы видели только в телевизоре.
"""

# Инициализация сплиттера
splitter = RecipeSplitter()

# Разбиение текста на чанки
chunks = splitter.split_text(text)
for i, chunk in enumerate(chunks, 1):
    print(f"Чанк {i}:\n{chunk}\n{'-' * 40}")

In [13]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

# Инициализация эмбеддингов
hf_embeddings_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2", #"cointegrated/LaBSE-en-ru",
    model_kwargs={"device": "cpu"}
)

from langchain_core.documents import Document

# Преобразование строк в объекты Document
documents = [Document(page_content=chunk) for chunk in chunks]

# Создание векторного хранилища
try:
    db_embed = FAISS.from_documents(
        documents, 
        hf_embeddings_model
    )
except Exception as e:
    raise RuntimeError(f"Ошибка при создании базы данных FAISS: {e}")

# Сохранение векторной базы локально
db_embed.save_local("../data/faiss_db_epub_mistral")

query = "ржаной виски, грейпфрутовый сок"
results = db_embed.similarity_search(query, k=1)  # Ищем 1 наиболее релевантный чанк
for result in results:
    print(result.page_content)